In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[909, 137, 166, 245, 856]

In [15]:
tweet_rdd = sc.textFile('./data/train.csv'). \
    map(lambda line: (line.split(',')[0], line.split(',')[-1]))

### Tweet Processing Functions

Below are the definitions of the functions that are used to process the tweets before being sent on to feature extraction

In [20]:
import re
from nltk.stem.porter import PorterStemmer

In [22]:
def basic_cleaning(tweet):
    clean_tweet = tweet.lower()
    clean_tweet = re.sub('[\.\?\,\!#]', '', clean_tweet)
    return clean_tweet